Build a code understanding model. Upload your own custom code files to the model and ask questions based on the code file as context.


Mount the google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Install Neccessary libraries

In [2]:
!pip install -q langchain openai tiktoken faiss-gpu pypdf rouge langchain-openai
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

Import libraries

In [3]:
from langchain.document_loaders import TextLoader
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [4]:
# Setting the OpenAI API key

import os
os.environ["OPENAI_API_KEY"] = "openai-api-key"



In [5]:
# Load and process the text file with code snippets

code_file_path = "/content/drive/MyDrive/code/Sample_code.txt"
loader = TextLoader(code_file_path)
code_snippets = loader.load_and_split()




In [6]:
# Split the code snippets into chunks

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(code_snippets)


In [7]:
# Create vector store

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(texts, embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
# Create an instance of the OpenAI language model

llm = OpenAI(temperature=0.1, model_name="gpt-4")

/usr/local/lib/python3.10/dist-packages/langchain_community/llms/openai.py:249: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_community/llms/openai.py:1070: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


In [9]:
# Create an instance of the RetrievalQA chain

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

In [ ]:
# Interactive question-answering loop

while True:
    question = input("Ask a question about the code (or type 'exit' to quit): ")
    if question.lower() == 'exit':
        break

    # Generate the answer using the RetrievalQA chain
    answer = qa_chain.run(question)

    print("Question:")
    print(question)
    print()
    print("Answer:")
    print(answer)
    print("---")

Part B: Write a chatbot prompt to iteratively create a sequence of chats on one particular custom data. 

In [18]:
!python -m pip install python-dotenv

In [19]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [21]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [23]:
def pdf_qa(pdf_path, chat_history):
    
    # Load the PDF document
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()

    # Split the text into chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)

    if not texts:
        print("No text chunks found in the PDF.")
        return

    # Create embeddings for the text chunks
    embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
    docsearch = FAISS.from_documents(texts, embeddings)

    # Load the question-answering chain
    chain = load_qa_chain(OpenAI(openai_api_key=OPENAI_API_KEY), chain_type="stuff")

    while True:
        query = input("Ask a question (type 'exit' to quit): ")
        if query.lower() == 'exit':
            break
        else:
            # Perform a similarity search to find the most relevant chunks
            docs = docsearch.similarity_search(query)

            # Run the question-answering chain
            answer = chain.run(input_documents=docs, question=query)
            print(answer)

            # Store the query and answer in the chat history
            chat_history.append(f"Human: {query}")
            chat_history.append(f"Assistant: {answer}")

    return chat_history



In [24]:
def summarise(chat_history):
    
    # Concatenate the chat history into a single string
    chat_text = "\n".join(chat_history)

    # Create a summary using the OpenAI API
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=f"Please provide a summary of the following chat conversation:\n\n{chat_text}\n\nSummary:",
        max_tokens=150,
        n=1,
        stop=None,
        temperature=0.1,
    )

    summary = response.choices[0].text.strip()
    return summary

In [16]:
# Give a PDF document; the chatbot answers the questions on the given PDF

pdf_path = "/content/drive/MyDrive/pdfs/10-Transformer-1.pdf"
chat_history = []
chat_history = pdf_qa(pdf_path, chat_history)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


Ask a question (type 'exit' to quit): What is this pdf about?
 This pdf is about the concept of attention and its application in the Transformer model in natural language processing.
Ask a question (type 'exit' to quit): What is a transformer?
 A transformer is a type of neural network used for natural language processing that incorporates both left and right context to create a more powerful context model. It also includes features such as attention and is split into an encoder and decoder. 
Ask a question (type 'exit' to quit): What is RNN?
 RNN is a Recurrent Neural Network, a type of neural model designed to process language and perform tasks such as classification, summarization, translation, and sentiment detection by using word embeddings and accessing previous words in the input sequence.
Ask a question (type 'exit' to quit): What is LSTM?
 LSTM is a type of recurrent neural network designed for processing language and performing tasks such as classification, summarization, tra

In [25]:
# Summarize the chats at the end of the conversation

summary = summarise(chat_history)
print("Chat Summary:")
print(summary)

Chat Summary:
The conversation discusses the concept of attention and its application in the Transformer model in natural language processing. It also explains the definitions of transformer, RNN, LSTM, softmax, self-attention, and key/value/query.
